In [1]:
import re
import pickle
import os
from IPython.display import display, HTML

In [2]:
from reader import Reader
src_file = "../../processed_pdfs/sample1/sample1.xml"
r = Reader({"src": src_file})
blocks, kv, text = r.get_content()
content = {"text": text, "blocks": blocks, "kv": kv}
with open("tmp.pkl", "wb") as fi:
    pickle.dump(content, fi)

Number of Pages:  2


In [3]:
from extractor import Extractor
M = Extractor("./definitions")

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
file_name = "tmp.pkl"
with open(file_name, "rb") as fi:
    content = pickle.load(fi)
M.set_content(content)
df = M.extract()
display(df)

,Name,Value,Page,Method
0,Inspection Date,2018,0,entity
1,Report Date,"11th January, 2018",0,entity
2,Report Type,Restricted Appraisal Report,0,lookup
3,Topography,Mostly level,1,text


In [18]:
for filename in files:
    file_path = os.path.join(tmp_dir, filename)
    if os.path.exists(file_path):
        with open(file_path, "rb") as fi:
            d = pickle.load(fi)
            texts = d["texts"]
            blocks = d["blocks"]
            kv = d["kv"]
            parser = Reader(params={"src": file_path})
            for item in defs:
                method = item["method"]
                name = item["name"]
                if method == "extract":
                    terms = item["search"]["terms"]
                    include = item["search"]["include"]
                    exclude = item["search"]["exclude"]
                    question = item["search"]["question"]
                    entities = item["types"]["entities"]
                    patterns = item["types"]["patterns"]        
                    c_patterns = [re.compile(p) for p in patterns]
                    pages_list = []
                    if "pages" in item["pages"] and item["pages"] is not None:
                        tmp = item["pages"].split("-")
                        if len(tmp) == 1:
                            pages_list = [int(tmp[0])]
                        elif len(tmp) == 2:
                            pages_list = range(int(tmp[0]), int(tmp[1]))
                    match_found = False
                    answer = None
                    for term in terms:
                        pages, kv_matches, block_matches = parser.search(term, blocks, kv, texts, include=include, exclude=exclude, pages_list=pages_list)
                        for p in pages:
                            page_kv = [tmp for tmp in kv_matches if tmp[0] == p]
                            page_block = [tmp for tmp in block_matches if tmp[0] == p]
                            for kv_item in page_kv:
                                pidx, bidx, data = kv_item
                                for d in data:
                                    key, value = d
                                    for pat in c_patterns:
                                        res = re.search(pat, value)
                                        if res is not None:
                                            match_found = True
                                            answer = value
                                            break
                                    if match_found:
                                        break
                                    if len(entities) == 0:
                                        match_found = True
                                        answer = value
                                        break
                                    doc = nlp(value)
                                    for ent in doc.ents:
                                        if ent.label_ in entities:
                                            answer = ent.text
                                            match_found = True
                                            break                            
                                    if match_found:
                                        break
                                if match_found:
                                    break
                            if match_found:
                                break
                            for block_item in page_block:
                                pidx, bidx, data = block_item
                                for d in data:
                                    value = qa_extractor.extract(question, d)
                                    for pat in c_patterns:
                                        res = re.search(pat, value)
                                        if res is not None:
                                            match_found = True
                                            answer = value
                                            break
                                    if match_found:
                                        break
                                    if len(entities) == 0:
                                        match_found = True
                                        answer = value
                                        break
                                    doc = nlp(value)
                                    for ent in doc.ents:
                                        if ent.label_ in entities:
                                            answer = ent.text
                                            match_found = True
                                            break                            
                                    if match_found:
                                        break                        
                            if match_found:
                                break
                        if match_found:
                            break
                    print (name, answer)
                elif method == "lookup":
                    pass

Inspection Date 20th January, 2018
Report Date December, 2017
Topography Level
